# Let’s use a simple example data to demonstrate how XGBoost algorithm works


### A) on a regression problem.

Let’s use a simple example data to demonstrate how XGBoost algorithm works on a regression problem.


#### Step 1. Set an initial prediction
Let’s set the initial prediction (F0(x)) to be 3. In other words, regardless of the value of X, the predicted Y will be 3 - mean value.


In [16]:
x = [5,8,15,20,-9]
y = [1,2,8,4,0]
pred_0 = sum(y) / len(y)
pred_0

3.0

In [18]:
X	|	Y   |	Pred_0(mean)
============================
5	|	1   |	3
8	|	2   |	3
15	|	8   |	3
20	|	4   |	3
-9 	|   0   |	3

SyntaxError: invalid syntax (999749795.py, line 2)

### Step 2. Calculate the residuals
In this step we calculate the difference between the actual value (Y) and the predicted value (F0(x)).


In [19]:
res_0 = []
for i in range(len(y)):
    res_0.append(y[i] - pred_0)
res_0

[-2.0, -1.0, 5.0, 1.0, -3.0]

### Step 3. Train model on the residuals

Here, we build one or more tree-based models that are trained on [X, res].

The following shows how a tree model is constructed.

#### A) Generate split candidates ####

In this step we find one or more split candidates. The formula is with N rows we would have N - 1 split candidates.

With our dataset example, here’s the result.

In [21]:
Raw NBConvert


===== Candidate Split 1 =====

X	|	Y   |Pred_0 |res_0 

============================
-9 	|   0   |	3   | -3
————————————————————————————————————————————————————————————————————————> threshold = (-9 + 5) / 2 = -2
5	|	1   |	3   | -2
8	|	2   |	3   | -1
15	|	8   |	3   |  5
20	|	4   |	3   |  1


===== Candidate Split 2 =====
X	|	Y   |Pred_0 |res_0 
============================
-9 	|   0   |	3   | -3
5	|	1   |	3   | -2
————————————————————————————————————————————————————————————————————————> threshold = (5 + 8) / 2 = 6.5
8	|	2   |	3   | -1
15	|	8   |	3   |  5
20	|	4   |	3   |  1

===== Candidate Split 3 =====

X	|	Y   |Pred_0 |res_0 
============================
-9 	|   0   |	3   | -3
5	|	1   |	3   | -2
8	|	2   |	3   | -1
————————————————————————————————————————————————————————————————————————> threshold = (8 + 15) / 2 = 11.5
15	|	8   |	3   |  5
20	|	4   |	3   |  1

===== Candidate Split 4 =====

X	|	Y   |Pred_0 |res_0 
============================
-9 	|   0   |	3   | -3
5	|	1   |	3   | -2
8	|	2   |	3   | -1
15	|	8   |	3   |  5
————————————————————————————————————————————————————————————————————————> threshold = (15 + 20) / 2 = 22.5
20	|	4   |	3   |  1





SyntaxError: invalid character '—' (U+2014) (975836391.py, line 10)

The threshold will be used to create two child nodes. 

Here’s the complete parent-child nodes for each candidate.

===== Candidate Split 1 =====

Parent node -> X < -2
Left child node, res_0 -> [-3]
Right child node, res_0 -> [-2, -1, 5, 1]

===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

===== Candidate Split 3 =====

Parent node -> X < 11.5
Left child node -> [-3, -2, -1]
Right child node -> [5, 1]

===== Candidate Split 4 =====

Parent node -> X < 22.5
Left child node -> [-3, -2, -1, 5]
Right child node -> [1]


#### B) Select the split
From step (A) we generate one or more split candidates. The question is how to select the best one?

In XGBoost, we select the best split by leveraging what are called as Similarity Score and Gain. Gain is calculated for the parent node, while Similarity Score is calculated for the child nodes. We select a candidate with the maximum Gain.

Here are the formulas.

Gain = Left_similarity_score + Right_similarity_score - Root_similarity_score

Similarity_score = SUM(residual_i)**2 / {SUM(prev_proba_i x (1 - prev_proba_i)) + lambda}

But in this case we will consider all possible trees with depth = 2

we have 
lamda = 1
gamma = 5




===== Candidate Split 1 =====

Parent node -> X < -2
Left child node, res_0 -> [-3]
Right child node, res_0 -> [-2, -1, 5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
SS_right = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
Gain = 4.5 + 1.8 - 0 - 5 = 0.3 > 0 => SPLITTING

    ___SPLIT1.1___
    
    left - nothing to split - 1 element
    right:
        ___SPLIT1.11___
        
        Parent node -> X < 6.5
        Left child node -> [-2]
        Right child node -> [-1, 5, 1] 
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2)**2 / (1 + 1)= 4/2 = 2
        SS_right = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        Gain = 2 + 6.25 - 1.8 - 5 = 1.45 > 0 => SPLITTING
        
        ___SPLIT1.12___

        Parent node -> X < 11.5
        Left child node -> [-2, -1]
        Right child node -> [5, 1]
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2 - 1)**2 / (2 + 1)= 9/3 = 3
        SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        Gain = 3 + 12 - 1.8 - 5 = 8.2 > 0 => SPLITTING
        
        ___SPLIT1.13___

        Parent node -> X < 22.5
        Left child node -> [-2, -1, 5]
        Right child node -> [1]
        
        SS_root = (-2 - 1 + 5 + 1)**2 / (4 + 1)= 9/5 = 1.8
        SS_left = (-2 - 1 + 5)**2 / (2 + 1)= 4/3 = 1.3
        SS_right = ( 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 1.3 + 0.5 - 1.8 - 5 = -5 < 0 => NO SPLITTING        
        

===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
SS_right = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
Gain = 8.3 + 6.25 - 0 - 5 = 9.58 > 0 => SPLITTING

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2]
        
        SS_root = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
        SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
        SS_right = (-2)**2 / (1 + 1)= 24/2 = 12
        Gain = 4.5 + 12 - 8.3 - 5 = 3.2 > 0 => SPLITTING
     

    right:
        ___SPLIT2.12___

        Parent node -> X < 11.5
        Left child node -> [-1]
        Right child node -> [5, 1]
        
        SS_root = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        SS_left = (- 1)**2 / (1 + 1)= 1/2 = 0.5
        SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        Gain = 0.5 + 12 - 6.25 - 5 = 1.25 > 0 => SPLITTING
        
        ___SPLIT1.13___

        Parent node -> X < 22.5
        Left child node -> [-1, 5]
        Right child node -> [1]
        
        SS_root = (- 1 + 5 + 1)**2 / (3 + 1)= 25/4 = 6.25
        SS_left = (- 1 + 5)**2 / (2 + 1)= 16/3 = 5.3
        SS_right = ( 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 5.3 + 0.5 - 6.25 - 5 = -5.45 < 0 => NO SPLITTING 

===== Candidate Split 3 =====

Parent node -> X < 11.5
Left child node -> [-3, -2, -1]
Right child node -> [5, 1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
SS_right = (5 + 1)**2 / (2 + 1)= 36/3 = 12
Gain = 9 + 12 - 0 - 5 = 16 > 0 => SPLITTING

    ___SPLIT3.1___
    
    left:
        ___SPLIT3.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2, -1]
        
        SS_root = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
        SS_left = (-3)**2 / (1 + 1)= 9/2 = 4.5
        SS_right = (-2 - 1)**2 / (2 + 1)= 9/3 = 3
        Gain = 4.5 + 3 - 9 - 5 = -5.5 <0 => NO SPLITTING
        
        ___SPLIT3.12___
        
        Parent node -> X < 6.5
        Left child node, res_0 -> [-3, -2]
        Right child node, res_0 -> [-1]
        
        SS_root = (-3 - 2 - 1)**2 / (3 + 1)= 36/4 = 9
        SS_left = (-3 - 2)**2 / (2 + 1)= 25/3 = 8.3
        SS_right = (- 1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 8.3 + 0.5 - 9 - 5 = -5.2 <0 => NO SPLITTING
        

    right:
        ___SPLIT3.13___

        Parent node -> X < 22.5
        Left child node -> [5]
        Right child node -> [1]
        
        SS_root = (5 + 1)**2 / (2 + 1)= 36/3 = 12
        SS_left = (5)**2 / (1 + 1)= 25/2 = 12.5
        SS_right = (1)**2 / (1 + 1)= 1/2 = 0.5
        Gain = 12.5 + 0.5 - 12 - 5 = -6 < 0 => NO SPLITTING 
    
        

===== Candidate Split 4 =====

Parent node -> X < 22.5
Left child node -> [-3, -2, -1, 5]
Right child node -> [1]

SS_root = (-3 -2 - 1 + 5 + 1)**2 / (5 + 1) = 0
SS_left = (-3 - 2 - 1 + 5)**2 / (4 + 1)= 1/5 = 0.20
SS_right = (1)**2 / (1 + 1)= 1/2 = 0.5
Gain = 0.20 + 0.5 - 0 - 5 = -4.3 < 0 => NO SPLITTING

Let's take one of the possible trees and take a look on the model
    


===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Right child node -> [-1, 5, 1]

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___       
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Right child node, res_0 -> [-2]

    right:
        ___SPLIT2.12___
        Parent node -> X < 11.5
        Left child node -> [-1]
        Right child node -> [5, 1]


        


#### c) Calculate Output Values
Output Values is calculated for all leaves of the decision tree.

The formula is given by the following:

In [22]:
Output Value = SUM(residual_i) / [SUM(n + lambda]

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (1725511801.py, line 1)


===== Candidate Split 2 =====

Parent node -> X < 6.5
Left child node -> [-3, -2]
Output  = (-3 - 2) / (2 + 1)= -5/3 = -1.6

Right child node -> [-1, 5, 1]
Output = (-1 + 5 -1) / (3 + 1)= 5/3 = 1.6

    ___SPLIT2.1___
    
    left:
        ___SPLIT2.11___
        
        Parent node -> X < -2
        Left child node, res_0 -> [-3]
        Output = (-3) / (1 + 1)= -3/2 = -1.5
        
        Right child node, res_0 -> [-2]
        Output = (-2) / (1 + 1)= -2/2 = -1
      
     
    right:
        ___SPLIT2.12___

        Parent node -> X < 11.5
        Left child node -> [-1]
        Output = (-1) / (1 + 1)= -1/2 = -0.5
        
        Right child node -> [5, 1]
        Output = (5 + 1) / (2 + 1)= 6/3 = 2

        

#### d) Predict the new values using our chosen DT

pred_1 = pred_0 + learning_rate * (DT_0(x))
Where: 
        pred_0 = 3 
        learning_rate = 0.1

### B) on a classification problem.

Let’s use a simple example data to demonstrate how XGBoost algorithm works on a regression problem.

In [23]:
X	|	Y
==========
-7	|	0
5	|	0
6	|	1
8	|	1
9 	|   1 

SyntaxError: invalid syntax (2444926364.py, line 2)

In [24]:
x = [-7, 5, 6, 8, 9]
y = [0, 0, 1, 1, 1]

#### Step 1. Set an initial prediction
Let’s set the initial prediction (F0(x)) to be 0.6. In other words, regardless of the value of X, the predicted Y will be 3 - mean value.

F(0)(mean) = (0 + 0 + 1 + 1 + 1)/ 5 = 0.6


In [25]:
X	|	Y   |	Pred_0(mean)
============================
-7	|	0   |	0.6
5	|	0   |	0.6
6	|	1   |	0.6
8	|	1   |	0.6
9 	|   1   |	0.6

SyntaxError: invalid syntax (1740534946.py, line 2)

### Step 2. Calculate the residuals and odds
In this step we calculate the difference between the actual value (Y) and the predicted value (F0(x)).

Also we will calculate odds according to next formula:
    odds = p/(1-p)
    
And then we will calculate logarithm of odds


In [26]:
res_0 = []
pred_0 = 0.6
for i in range(len(y)):
    res_0.append(y[i] - pred_0)
res_0

[-0.6, -0.6, 0.4, 0.4, 0.4]

In [27]:
X	|	Y   |	Pred_0(mean) |res_0 |odds_0 |log(odds_0)
========================================================
-7	|	0   |	    0.6      |-0.6  | 1.5   | 0.405
5	|	0   |	    0.6      |-0.6  | 1.5   | 0.405
6	|	1   |	    0.6      | 0.4  | 1.5   | 0.405
8	|	1   |	    0.6      | 0.4  | 1.5   | 0.405
9 	|   1   |	    0.6      | 0.4  | 1.5   | 0.405

SyntaxError: invalid syntax (3697008290.py, line 2)

### Step 3. Train model on the residuals

Here, we build one or more tree-based models that are trained on [X, res].

The following shows how a tree model is constructed.

#### A) Generate split candidates ####
In this step we find one or more split candidates. The formula is with N rows we would have N - 1 split candidates.
With our dataset example, here’s the result.

In [28]:
Raw NBConvert

== == = Candidate Split 1 == == =

X	|	Y   |	Pred_0(mean) |res_0
===================================
-7	|	0   |	    0.6      |-0.6
———————————————————————————————————————————————————————————————————————— > threshold = (-7 + 5) / 2 = 1
5	|	0   |	    0.6      |-0.6
6	|	1   |	    0.6      | 0.4
8	|	1   |	    0.6      | 0.4
9 	|   1   |	    0.6      | 0.4

== == = Candidate Split 2 == == =

X	|	Y   |	Pred_0(mean) |res_0
===================================
-7	|	0   |	    0.6      |-0.6
5	|	0   |	    0.6      |-0.6
———————————————————————————————————————————————————————————————————————— > threshold = (5 + 6) / 2 = 5.5
6	|	1   |	    0.6      | 0.4
8	|	1   |	    0.6      | 0.4
9 	|   1   |	    0.6      | 0.4


== == = Candidate Split 3 == == =

X	|	Y   |	Pred_0(mean) |res_0
===================================
-7	|	0   |	    0.6      |-0.6
5	|	0   |	    0.6      |-0.6
6	|	1   |	    0.6      | 0.4
———————————————————————————————————————————————————————————————————————— > threshold = (6 + 8) / 2 = 7
8	|	1   |	    0.6      | 0.4
9 	|   1   |	    0.6      | 0.4


== == = Candidate Split 4 == == =

X	|	Y   |	Pred_0(mean) |res_0
===================================
-7	|	0   |	    0.6      |-0.6
5	|	0   |	    0.6      |-0.6
6	|	1   |	    0.6      | 0.4
8	|	1   |	    0.6      | 0.4
———————————————————————————————————————————————————————————————————————— > threshold = (8 + 9) / 2 = 8.5
9 	|   1   |	    0.6      | 0.4



SyntaxError: invalid character '—' (U+2014) (263720272.py, line 8)

The threshold will be used to create two child nodes.

Here’s the complete parent-child nodes for each candidate.

In [29]:
===== Candidate Split 1 =====

Parent node -> X < 1
Left child node, res_0 -> [-0.6]
Right child node, res_0 -> [-0.6, 0.4, 0.4, 0.4]

===== Candidate Split 2 =====

Parent node -> X < 5.5
Left child node -> [-0.6, -0.6]
Right child node -> [0.4, 0.4, 0.4]

===== Candidate Split 3 =====

Parent node -> X < 7
Left child node -> [-0.6, -0.6, 0.4]
Right child node -> [0.4, 0.4]

===== Candidate Split 4 =====

Parent node -> X < 8.5
Left child node -> [-0.6, -0.6, 0.4, 0.4]
Right child node -> [0.4]


SyntaxError: invalid syntax (119867641.py, line 1)

#### B) Select the split
From step (A) we generate one or more split candidates. 

In XGBoost, we select the best split by leveraging what are called as Similarity Score and Gain. Gain is calculated for the parent node, while Similarity Score is calculated for the child nodes. We select a candidate with the maximum Gain.

Here are the formulas.

Gain = Left_similarity_score + Right_similarity_score - Root_similarity_score

Similarity_score = SUM(residual_i)**2 / {SUM(prev_proba_i x (1 - prev_proba_i)) + lambda}

For our example we  will take 1 random tree with depth = 2

we have 
lamda = 1
gamma = 0.2


== == = Candidate Split 2 == == =

Parent node -> X < 5.5
Left child node -> [-0.6, -0.6]
Right child node -> [0.4, 0.4, 0.4]

SS_root = (-0.6 - 0.6 + 0.4 + 0.4 + 0.4)**2 / (0.6*0.4 + 0.6*0.4 + 0.4*0.6 + 0.4*0.6 + 0.4*0.6  + 1) = 0
SS_left = (-0.6 - 0.6)**2 / (2*(0.6*0.4) + 1)= 1.44/1.48 = 0.97
SS_right = (0.4 + 0.4 + 0.4)**2 / (3*(0.6*0.4) + 1)= 1.44/1.72 = 0.83
Gain = 0.97 + 0.83 - 0 - 0.2 = 1.6 > 0 => SPLITTING

    ___SPLIT2.1___
    
    left:
    ___SPLIT2.11___
    
    Parent node -> X < 1
    Left child node, res_0 -> [-0.6]
    Right child node, res_0 -> [-0.6]
    
    SS_root = (-0.6 - 0.6)**2 / (2*(0.6*0.4) + 1)= 1.44/1.48 = 0.97
    SS_left = (-0.6)**2 / ((0.6*0.4) + 1)= 0.36/1.24 = 0.29
    SS_right = (-0.6)**2 / ((0.6*0.4) + 1)= 0.36/1.24 = 0.29
    Gain = 0.29 + 0.29 - 0.97 - 0.2  = -0.59 < 0 => NO SPLITTING

    right:
    ___SPLIT2.12___
    
    Parent node -> X < 7
    Left child node -> [0.4]
    Right child node -> [0.4, 0.4]
    
    SS_root = (0.4 + 0.4 + 0.4)**2 / (3*(0.6*0.4) + 1)= 1.44/1.72 = 0.83
    SS_left = (0.4)**2 / ((0.6*0.4) + 1)= 0.16/1.24 = 0.12
    SS_right = (0.4 + 0.4)**2 / (2*(0.6*0.4) + 1)= 0.64/1.48 = 0.43
    Gain = 0.12 + 0.43 - 0.83 - 0.2  = -0.48 < 0 => NO SPLITTING
    
    right:
    ___SPLIT2.13___
    
    Parent node -> X < 8.5
    Left child node -> [0.4, 0.4]
    Right child node -> [0.4]
    
    calculations symmetrical to ___SPLIT2.12___
    
    SS_root = (0.4 + 0.4 + 0.4)**2 / (3*(0.6*0.4) + 1)= 1.44/1.72 = 0.83
    SS_left = (0.4)**2 / ((0.6*0.4) + 1)= 0.16/1.24 = 0.12
    SS_right = (0.4 + 0.4)**2 / (2*(0.6*0.4) + 1)= 0.64/1.48 = 0.43
    Gain = 0.12 + 0.43 - 0.83 - 0.2  = -0.48 < 0 => NO SPLITTING


#### c) Calculate Output Values
Output Values is calculated for all leaves of the decision tree.

The formula is given by the following:
Output Value = SUM(residual_i) / [SUM(prev_proba_i x (1 - prev_proba_i)) + lambda]


== == = Candidate Split 2 == == =

Parent node -> X < 5.5
Left child node -> [-0.6, -0.6]
Right child node -> [0.4, 0.4, 0.4]

Output_left = (-0.6 - 0.6) / (2*(0.6*0.4) + 1)= -1.2/1.48 = -0.81
Output_right = (0.4 + 0.4 + 0.4) / (3*(0.6*0.4) + 1)= 1.2/1.72 = 0.69


#### d) Predict the new values using our chosen DT

To predict the new values, we use the following formula.

log(odds)_F1(x) = log(odds)_F0(x) + learning_rate * (DT_0(x))

F1(x) is retrieved by computing e^log(odds)_F1(x) / [1 + e^log(odds)_F1(x)]

Let’s apply this step to our dataset. Assume that learning_rate is set to 0.1.

In [30]:
log(odds)_F1(-7) = 0.405 + 0.1*(-0.81) = - 0.324
log(odds)_F1(5) = 0.405 + 0.1*(-0.81) = - 0.324
log(odds)_F1(6) = 0.405 + 0.1*(0.69) = 0.474
log(odds)_F1(8) = 0.405 + 0.1*(0.69) = 0.474
log(odds)_F1(9) = 0.405 + 0.1*(0.69) = 0.474

F1(-7) = exp**(- 0.324)/ (1 + exp**(- 0.324) = 0.419
F1(5) = exp**(0.474)/ (1 + exp**(0.474) = 0.616
F1(6) = exp**(0.474)/ (1 + exp**(0.474) = 0.616
F1(8) = exp**(0.474)/ (1 + exp**(0.474) = 0.616
F1(9) = exp**(0.474)/ (1 + exp**(0.474) = 0.616

SyntaxError: invalid syntax (9164966.py, line 1)

In [31]:
X	|	Y   |	Pred_0(mean) |res_0 |odds_0 |log(odds_0)|Pred_1
======================================================================
-7	|	0   |	    0.6      |-0.6  | 1.5   | 0.405     | 0.419
5	|	0   |	    0.6      |-0.6  | 1.5   | 0.405     | 0.419
6	|	1   |	    0.6      | 0.4  | 1.5   | 0.405     | 0.616
8	|	1   |	    0.6      | 0.4  | 1.5   | 0.405     | 0.616
9 	|   1   |	    0.6      | 0.4  | 1.5   | 0.405     | 0.616

SyntaxError: invalid syntax (872557540.py, line 2)